In [8]:
import sqlite3

In [ ]:
import sqlite3

conn = sqlite3.connect('mysqlite.db')
c = conn.cursor()
#records or rows in a list
records = [(1, 'Glen', 8),
			(2, 'Elliot', 9),
			(3, 'Bob', 7)]

#insert multiple records in a single query
c.executemany('INSERT INTO students VALUES(?,?,?);',records);

print('We have inserted', c.rowcount, 'records to the table.')

#commit the changes to db			
conn.commit()
#close the connection
conn.close()


In [1]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""SELECT static_id ,page_name FROM ads """
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
cursor.close()
# records = [i[0] for i in records]
print(len(records))
for record in records:
    try:
        cursor.execute('''INSERT INTO facebook_pages VALUES 
        (?, ?,?)'''
                  , (record[0],record[1],False))
        sqliteConnection.commit()
    except:
        pass

8319


In [97]:
# def get_records(SLQQuery):
    sqliteConnection = sqlite3.connect('FaceBoookADS.db')
    cursor = sqliteConnection.cursor()
    cursor.execute(SLQQuery)
    records = cursor.fetchall()
#     print(len(records))
    sqliteConnection.close()
    return records

In [69]:
# sqlite_select_query = f"""SELECT DISTINCT static_id,page_name, FROM ads where page_name is not '' """
# static_ids = [i[0] for i in get_records(sqlite_select_query)] 
# static_ids.remove('NO static ID found')
# # records = [i[0] for i in records]
# # print(len(records))
# # for record in records:
# #     try:
# #         cursor.execute('''INSERT INTO facebook_pages VALUES 
# #         (?, ?,?)'''
# #                   , (record[0],record[1],False))
# #         sqliteConnection.commit()
# #     except:
# #         pass

4699


### create pages_datebase



In [70]:
sqlite_select_query = f"""SELECT DISTINCT static_id FROM ads where page_name is not '' """
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
sqliteConnection.close()
static_ids = [i[0] for i in records] 
static_ids.remove('NO static ID found')
for static_id in static_ids:
    sqliteConnection = sqlite3.connect('FaceBoookADS.db')
    cursor = sqliteConnection.cursor()
    sqlite_select_query = f"""SELECT ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name FROM ads where static_id = {static_id} order by date desc limit 1"""
    cursor.execute(sqlite_select_query)
    records = cursor.fetchall()
    ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name  = records[0]
    cursor.close()
    sqliteConnection.close()
    try:
        sqliteConnection = sqlite3.connect('FaceBoookADS.db')
        cursor = sqliteConnection.cursor()
        cursor.execute(f"""insert into facebook_pages(static_ID,ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name,favorites,block) VALUES (?,?,?,?,?,?,?,?,0,0)""",(static_id,ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name))
        sqliteConnection.commit()
        sqliteConnection.close()
    except:
        sqliteConnection = sqlite3.connect('FaceBoookADS.db')
        cursor = sqliteConnection.cursor()
        cursor.execute(f"""
        update facebook_pages set 
        Ads_count = ?,
        Facebook_ID = ?,
        insta_followers = ?,
        instgram_ID = ?,
        Page_likes = ?,
        profile_pic = ?,
        page_name = ?
        where static_id = ?
        """,(ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name,static_id))
        sqliteConnection.commit()
        sqliteConnection.close()
#clean_old_pages 
sqlite_select_query = f"""DELETE FROM facebook_pages where Ads_count is NULL """
#DELETE FROM table
# WHERE search_condition;
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
cursor.execute(sqlite_select_query)
sqliteConnection.commit()
# records = cursor.fetchall()
sqliteConnection.close()

4692


## solve no data problem 

## count

In [162]:
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
cursor.execute("""select static_ID, count(*) as c FROM Ads GROUP BY static_ID""")
records = cursor.fetchall()
#     print(len(records))
sqliteConnection.close()

In [163]:
records

[('0', 1),
 ('100100119547239', 1),
 ('100101471794502', 2),
 ('100106772530451', 1),
 ('100109449544319', 1),
 ('100109651405241', 1),
 ('100110425921069', 13),
 ('100111301813165', 2),
 ('100112392883547', 3),
 ('100113002863377', 2),
 ('100116466071014', 5),
 ('100118665230213', 1),
 ('100119601914982', 2),
 ('100119731969906', 1),
 ('100121389440517', 1),
 ('100123525776787', 1),
 ('100124119232558', 1),
 ('100124741443742', 5),
 ('100125292366804', 1),
 ('100127202656038', 2),
 ('100128456218462', 2),
 ('100129335253816', 1),
 ('100132049394692', 1),
 ('100135505645208', 3),
 ('100136379150137', 1),
 ('100136521449570', 1),
 ('100136819540297', 1),
 ('100138959552683', 1),
 ('100147186159541', 2),
 ('100150285649670', 2),
 ('100151212798459', 1),
 ('100154388439386', 1),
 ('100154432044067', 1),
 ('100155896134144', 1),
 ('100159685363036', 2),
 ('100160619446323', 1),
 ('1001619056584927', 1),
 ('100164375526308', 2),
 ('100166622763048', 1),
 ('100170209367620', 1),
 ('100171215